In [40]:

build ="""
const std = @import("std");
const Builder = std.build.Builder;
const Root = std.build.Root;

pub fn build(b: *Builder) void {
    const mode = b.standardReleaseOptions();

    const main_exe = b.addExecutable(Root.main_exe(.{
        .name = "zig_chat_bot",
        .root_source_file = "main.zig",
        .target = b.standardTargetOptions(),
        .optimize = mode,
    }));

    // Add include path for C source files
    main_exe.addIncludeDir("c-src");

    // Link against libc
    main_exe.linkLibC();

    const result = b.build();
    if (result != .Success) {
        std.debug.print("Build failed: {}\n", .{result});
        return;
    }
}
"""

with open('build.zig', 'w') as f:
    f.write(build)

In [25]:
# c_utils

c_utils= """

#include <stdio.h>
#include <dirent.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <errno.h>

// Function to count the number of files in a directory
int count_files(const char *path) {
    DIR *dr = opendir(path);
    if (dr == NULL) {
        printf("Couldn't open DIR Error No: %d\n", errno);
        return -1;
    }
    
    struct dirent *de;
    int num_files = 0;

    while ((de = readdir(dr)) != NULL) {
        if (strcmp(de->d_name, ".") == 0 || strcmp(de->d_name, "..") == 0) {
            continue;
        }
        num_files++;
    }
    closedir(dr);

    int32_t num_files32 = (int32_t)num_files;
    return num_files32;
}

// Function to get the names of files in a directory
char **get_file_names(const char *path, int num_files) {
    struct dirent *de;
    DIR *dr = opendir(path);
    if (dr == NULL) {
        perror("Couldn't open dir");
        return NULL;
    }

    char **file_names = (char **)malloc(num_files * sizeof(char *));
    if (file_names == NULL) {
        perror("Memory allocation failed");
        closedir(dr);
        return NULL;
    }

    int index = 0;
    while ((de = readdir(dr)) != NULL) {
        if (strcmp(de->d_name, ".") == 0 || strcmp(de->d_name, "..") == 0) {
            continue;
        }
        file_names[index] = (char *)malloc(strlen(de->d_name) + 1);
        if (file_names[index] == NULL) {
            perror("Memory allocation failed");
            // Free previously allocated memory
            for (int i = 0; i < index; i++) {
                free(file_names[i]);
            }
            free(file_names);
            closedir(dr);
            return NULL;
        }
        strcpy(file_names[index], de->d_name);
        printf("file_name_now: %s\n", file_names[index]);
        index++;
    }
    closedir(dr);
    return file_names;
}



"""

with open('c_utils.c', 'w') as f:
    f.write(c_utils)

In [27]:
! gcc -c c_utils.c -o c_utils.o

In [17]:
# utils.zig
utils = """
const std = @import("std");
pub fn add(a : i32, b : i32) i32
{
    return a + b;
}

// Define a helper function
pub fn cStringTozigSttring(ptr: [*]const u8) usize 
{
    var index : usize = 0;
    while (ptr[index] != 0) : (index += 1) {}
    return index;
}
"""

with open('utils.zig', 'w') as f:
    f.write(utils)

In [28]:
!ls

chatbot.ipynb  c_utils.o  main	  main.zig
c_utils.c      files.zig  main.o  utils.zig


In [19]:
# files.zig
files = """
const std = @import("std");
const utils = @import("utils.zig");
//const get_path =  @import("get_path.zig");

const files = @cImport
    (
        {
            @cInclude("c_utils.c");
            @cInclude("stdio.h");
        }
    );

pub fn read_file_names() [*c][*c]u8 {
    const path :[*c]const u8 = "home/albin/Downloads/zig-linux-x86_64-0.14.0-dev.185+c40708a2c/workflow_chatbot";
    const num_files = files.count_files(path);
    const filenames : [*c][*c]u8 = files.get_file_names(path, num_files);
    return filenames;
}
"""

with open('files.zig', 'w') as f:
    f.write(files)

In [55]:
# main.zig
main_code = """
const std = @import("std");

const utils = @import("utils.zig");

const files = @import("files.zig");

pub fn main() void {
    const result = utils.add(5, 3);
    std.debug.print("The result is {d}\\n", .{result});

    const filesFrompath = files.read_file_names();
    std.log.info("the files are : {s}", .{filesFrompath[0..4]});
}
"""

with open('main.zig', 'w') as f:
    f.write(main_code)

In [21]:
import os
print(os.environ['PATH'])

/home/albin/Downloads/zig-linux-x86_64-0.14.0-dev.185+c40708a2c
:zig:/home/albin/anaconda3/envs/jupyter_zig_env/bin:/home/albin/anaconda3/condabin:/home/albin/.local/bin:/home/albin/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/snap/bin:/home/albin/Downloads/zig-linux-x86_64-0.14.0-dev.185+c40708a2c


In [4]:
import os
#os.environ['PATH'] += ":/home/albin/Downloads/zig-linux-x86_64-0.14.0-dev.185+c40708a2c"

In [ ]:
#!echo $CONDA_PREFIX
print(os.listdir(os.environ.get('CONDA_PREFIX')+'/lib'))

In [ ]:
echo $CONDA_PREFIX/lib
print(os.environ.get('CONDA_PREFIX'))

In [11]:
!which gcc

/usr/bin/gcc


In [12]:
!which ld

/usr/bin/ld


In [2]:
!find $CONDA_PREFIX -name "libc.so*"

/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/lib64/libc.so.6
/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/usr/lib64/libc.so


In [6]:
!find $CONDA_PREFIX -name "libdl.so*"

/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/lib64/libdl.so.2
/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/usr/lib64/libdl.so


In [7]:
!find $CONDA_PREFIX -name "crt1.o"

/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/usr/lib64/crt1.o


In [17]:
!find $CONDA_PREFIX -name "libm.so*"

/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/lib64/libm.so.6
/home/albin/anaconda3/envs/jupyter_zig_env/x86_64-conda-linux-gnu/sysroot/usr/lib64/libm.so


In [56]:
#compile zig code
!../zig build-exe main.zig -femit-bin=main -freference-trace=5 -target native-native-gnu -I . -lc -ldl -lrt

[1865] Semantic Analysis
(0tq(B hash.crc.impl.Crc(u32,.{ .polynomial = 7
(0mq(B hash.crc.impl.Crc(u32,.{ .polynomial = 7
[237] Code Generation
MMMM[2070] Semantic Analysis
(0mq(B debug.dumpStackTraceFromBase
[399] Code Generation
MMM[2487] Semantic Analysis
(0mq(B fmt.format__anon_10736
[559] Code Generation
MMM[2703] Semantic Analysis
(0mq(B mem.Allocator.free__anon_11017
[808] Code Generation
MMMLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object
MLLVM Emit Object


In [57]:
#Run compiled zig progamm and capture output
!./main

The result is 8
info: the files are : { main.o, main, build.zig, chatbot.ipynb }


In [42]:
!cat build.zig


const std = @import("std");
const Builder = std.build.Builder;
const Root = std.build.Root;

pub fn build(b: *Builder) void {
    const mode = b.standardReleaseOptions();

    const main_exe = b.addExecutable(Root.main_exe(.{
        .name = "zig_chat_bot",
        .root_source_file = "main.zig",
        .target = b.standardTargetOptions(),
        .optimize = mode,
    }));

    // Add include path for C source files
    main_exe.addIncludeDir("c-src");

    // Link against libc
    main_exe.linkLibC();

    const result = b.build();
    if (result != .Success) {
        std.debug.print("Build failed: {}
", .{result});
        return;
    }
}


In [60]:
!ls 

build.zig      c_utils.c  files.zig  main.o    utils.zig
chatbot.ipynb  c_utils.o  main	     main.zig


In [61]:
!./main

The result is 8
